In [7]:
%pylab nbagg
import sys
sys.path.append('./tvb-library/')
sys.path.append('./tvb-data/')
# sys.path.append('/Users/adam2392/Documents/fragility_analysis/')
# import fragility.signalprocessing as sp

from tvb.simulator.lab import *
import os.path
from matplotlib import colors, cm
import time
import scipy.signal as sig

import numpy as np
import pandas as pd
import scipy.io

# downloaded library for peak detection in z time series
import peakdetect
from runmainsim import *
import tvbsim

import re

import ipyparallel as ipp

import multiprocessing as mp
from contextlib import closing
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
rc = ipp.Client()
dv = rc[:]
lbv = rc.load_balanced_view()

print rc
print dv
print lbv

<DirectView [0, 1, 2, 3]>
<LoadBalancedView None>


In [13]:
def runclustersim(i):
    patient = toShare[i]
    print patient

    MOVECONTACT = 1

    # 1000 = 1 second
    samplerate = 1000 # Hz
    sim_length = 240*samplerate    
    period = 1

    ######### Region Parameters ##########
#     ezregion = ['ctx-lh-bankssts', 'ctx-lh-cuneus']
    ezregion = ['ctx-lh-bankssts']
#     ezregion = []
#     pzregion = ['ctx-lh-cuneus']
    pzregion = []

    ######### Epileptor Parameters ##########
    # intialized hard coded parameters
    epileptor_r = 0.0002       # Temporal scaling in the third state variable
    epiks = -15               # Permittivity coupling, fast to slow time scale
    epitt = 0.05               # time scale of simulation
    epitau = 10                # Temporal scaling coefficient in fifth st var
    # x0c value = -2.05
    x0norm=-2.5
    x0ez=-1.6
    x0pz=-2.04

    # depends on epileptor variables of interest: it is where the x2-y2 var is
    varindex = [1]

    ######### Integrator Parameters ##########
    # parameters for heun-stochastic integrator
    heun_ts = 0.05
    noise_cov = np.array([0.001, 0.001, 0.,\
                    0.0001, 0.0001, 0.])

    root_dir = os.path.join('/home/adamli/')
    project_dir = os.path.join(root_dir, "metadata/",patient)
    tvbsim.util.renamefiles(patient, project_dir)

    ####### Initialize files needed to 
    # convert seeg.xyz to seeg.txt file
    sensorsfile = os.path.join(project_dir, "seeg.txt")
    # convert gain_inv-square.mat file into gain_inv-square.txt file
    gainmatfile = os.path.join(project_dir, "gain_inv-square.txt")

    confile = os.path.join(project_dir, "connectivity.zip")
    ####################### 1. Structural Connectivity ########################
    con = initconn(confile)

    # extract the seeg_xyz coords and the region centers
    seeg_xyz = tvbsim.util.extractseegxyz(sensorsfile)
    seeg_labels = seeg_xyz.index.values
    region_centers = con.centres
    regions = con.region_labels
    num_regions = len(regions)

    # initialize object to assist in moving seeg contacts
    movecontact = tvbsim.util.MoveContacts(seeg_labels, seeg_xyz, 
                                       regions, region_centers, True)

    if MOVECONTACT:
        ezindices = movecontact.getindexofregion(ezregion)
        pzindices = movecontact.getindexofregion(pzregion)
        
        ########## MOVE INDICES
        # move electrodes onto ez indices
        elecmovedindices = []
        for ezindex in ezindices:
            print "Moving onto current ez index: ", ezindex, " at ", regions[ezindex]
             # find the closest contact index and distance
            seeg_index, distance = movecontact.findclosestcontact(ezindex, elecmovedindices)

            # get the modified seeg xyz and gain matrix
            modseeg, electrodeindices = movecontact.movecontact(ezindex, seeg_index)
            elecmovedindices.append(electrodeindices)
        
        # use subcortical structures!
        use_subcort = 1
        verts, normals, areas, regmap = tvbsim.util.read_surf(metadatadir, use_subcort)
        modgain = tvbsim.util.gain_matrix_inv_square(verts, areas,
                            regmap, len(region_labels), movecontact.seeg_xyz)
        print "modified gain matrix the TVB way!"
        # simplest gain matrix will result in an error due to moving electrode 
        # directly onto an ez region, or any region for that matter
#         modgain = movecontact.simplest_gain_matrix(movecontact.seeg_xyz)
    elif MOVECONTACT == 0:
        ########## SET EZREGION BASED ON CLOSENESS
        # find the closest region-contact pair
        for idx, label in enumerate(seeg_labels):
            region_index, distance = movecontact.getregionsforcontacts(label)

            if idx == 0:
                mindist = distance
                minregion = region_index
                mincontact = label
            else:
                if distance < mindist:
                    mindist = distance
                    minregion = region_index
                    mincontact = label
                    
        ezregion = minregion
#         ezregion = np.append(ezregion,'ctx-lh-cuneus')
    ezindices = movecontact.getindexofregion(ezregion)
    pzindices = movecontact.getindexofregion(pzregion)
        
    outputdir = os.path.join('/home/adamli/data/tvbforwardsim/', patient)
    if not os.path.exists(outputdir):
        os.mkdir(outputdir)
    filename = os.path.join(outputdir, patient+'_sim_nez'+str(len(ezregion))+\
                                        '_npz'+str(len(pzregion))+'.npz')
    print "Project directory for meta data is : ", project_dir
    print "File to be saved is: ", filename
    
    ####################### 2. Neural Mass Model @ Nodes ######################
    epileptors = initepileptor(epileptor_r, epiks, epitt, epitau, x0norm, \
                              x0ez, x0pz, ezindices, pzindices, num_regions)    
    ####################### 3. Integrator for Models ##########################
    heunint = initintegrator(heun_ts, noise_cov)
    ################## 4. Difference Coupling Between Nodes ###################
    coupl = initcoupling(a=1.)
    ############## 5. Import Sensor XYZ, Gain Matrix For Monitors #############
    monitors = initmonitors(period, sensorsfile, gainmatfile, varindex)

    if MOVECONTACT:
        print "moving contacts for ", patient
        # modify the config of the monitors
        monitors[1].sensors.locations = movecontact.seeg_xyz
        monitors[1].gain = modgain

    # get initial conditions and then setup entire simulation configuration
    initcond = initconditions(x0norm, num_regions)
    sim, configs = setupconfig(epileptors, con, coupl, heunint, monitors, initcond)

    times, epilepts, seegts = runsim(sim, sim_length)

    postprocessor = tvbsim.util.PostProcess(epilepts, seegts, times)
    ######################## POST PROCESSING #################################
    # post process by cutting off first 5 seconds of simulation
    # for now, don't, since intiial conditions
    times, epits, seegts, zts = postprocessor.postprocts(samplerate)

    # get the onset, offset times
    onsettimes = None
    offsettimes = None
    try:
        onsettimes, offsettimes = postprocessor.findonsetoffset(zts[ezindices, :].squeeze(), delta=0.2)
    except:
        print "Still not working..."

    ######################## SAVING ALL DATA #################################
    regions = configs.connectivity.region_labels
    # Save files
    meta = {
        'x0ez':x0ez,
        'x0pz':x0pz,
        'x0norm':x0norm,
        'regions': regions,
        'regions_centers': configs.connectivity.centres,
        'seeg_contacts': configs.monitors[1].sensors.labels,
        'seeg_xyz': configs.monitors[1].sensors.locations,
        'ez': regions[ezindices],
        'pz': regions[pzindices],
        'ezindices': ezindices,
        'pzindices': pzindices,
        'onsettimes':onsettimes,
        'offsettimes':offsettimes,
        'patient':patient,
    }

    # save tseries
    np.savez(filename, epits=epits, seegts=seegts, \
             times=times, zts=zts, metadata=meta)

In [14]:
patients = ['id001_ac', 'id002_cj', 'id014_rb']
indices = np.arange(0, len(patients))

manager = mp.Manager()
toShare=  manager.list(patients)

with closing(mp.Pool()) as p:
    results = p.map_async(runclustersim, indices)

id002_cj
id014_rb
id001_ac
Already renamed seeg.xyz possibly!
Already renamed seeg.xyz possibly!
Already renamed seeg.xyz possibly!
Already renamed gain_inv-square.mat possibly!
Already renamed gain_inv-square.mat possibly!
Already renamed gain_inv-square.mat possibly!
Moving onto current ez index:  0  at  ctx-lh-bankssts
OR'1
OR
[-17.97  -4.23   4.62]


 movecontact function summary: 

Closest contact to ezregion:  [-46.4062 -68.0003  14.9149]  is  OR'1
Moving onto current ez index:  0  at  ctx-lh-bankssts
That is located at:  [-46.4062 -68.0003  14.9149]
It will move:  [-28.436199999999999, -63.770299999999992, 10.294899999999998]
Moving onto current ez index:  0  at  ctx-lh-bankssts
New location after movement is [-46.4062 -68.0003  14.9149]


H'13

H
GPH'4
GPH
[-47.69 -17.06   0.9 ]


 movecontact function summary: 
[-44.62 -36.16 -22.87]



 movecontact function summary: 

Closest contact to ezregion:  [-45.3305 -31.7053  -4.7786]  is  H'13
Closest contact to ezregion:  [-47.4046 

In [15]:
print results
print results.get()

AttributeError: 'module' object has no attribute 'read_surf'

# 1. Set Up Simulation Configuration First - Move Contact
Here, set everything from the ez region, connectivity, node model, coupling, monitors, integrator, and parameters.

In [ ]:
patients = ['id001_ac', 'id002_cj', 'id014_rb']
MOVECONTACT = 1

# 1000 = 1 second
samplerate = 1000 # Hz
sim_length = 240*samplerate    
period = 1

######### Region Parameters ##########
ezregion = ['ctx-lh-bankssts', 'ctx-lh-cuneus']
#     ezregion = ['ctx-lh-bankssts']
#     ezregion = []
pzregion = ['ctx-lh-cuneus']
pzregion = []
    
######### Epileptor Parameters ##########
# intialized hard coded parameters
epileptor_r = 0.0002       # Temporal scaling in the third state variable
epiks = -10               # Permittivity coupling, fast to slow time scale
epitt = 0.05               # time scale of simulation
epitau = 10                # Temporal scaling coefficient in fifth st var
# x0c value = -2.05
x0norm=-2.5
x0ez=-1.6
x0pz=-2.04

# depends on epileptor variables of interest: it is where the x2-y2 var is
varindex = [1]

######### Integrator Parameters ##########
# parameters for heun-stochastic integrator
heun_ts = 0.05
noise_cov = np.array([0.001, 0.001, 0.,\
                0.0001, 0.0001, 0.])
    
# patients = ['id001_ac']
for patient in patients:
    print patient

    outputdir = os.path.join('/Users/adam2392/Documents/pydata/tvb_forwardsim/', patient)
    if not os.path.exists(outputdir):
        os.mkdir(outputdir)
    filename = os.path.join(outputdir, patient+'_nonmovedsim_nez'+str(len(ezregion))+\
                                        '_npz'+str(len(pzregion))+'.npz')
    
    root_dir = os.getcwd()
    project_dir = os.path.join(root_dir, "metadata/"+patient)
    print "Project directory for meta data is : ", project_dir
    print "File to be saved is: ", filename
    renamefiles(patient, project_dir)
    
    ####### Initialize files needed to 
    # convert seeg.xyz to seeg.txt file
    sensorsfile = os.path.join(project_dir, "seeg.txt")
    # convert gain_inv-square.mat file into gain_inv-square.txt file
    gainmatfile = os.path.join(project_dir, "gain_inv-square.txt")
    
    confile = os.path.join(project_dir, "connectivity.zip")
    ####################### 1. Structural Connectivity ########################
    con = initconn(confile)
    
    # extract the seeg_xyz coords and the region centers
    seeg_xyz = tvbsim.util.extractseegxyz(sensorsfile)
    seeg_labels = seeg_xyz.index.values
    region_centers = con.centres
    regions = con.region_labels
    num_regions = len(regions)
    
    # initialize object to assist in moving seeg contacts
    movecontact = tvbsim.util.MoveContacts(seeg_labels, seeg_xyz, 
                                       regions, region_centers, True)
    
    ezindices = movecontact.getindexofregion(ezregion)
    pzindices = movecontact.getindexofregion(pzregion)
    
    # move electrodes onto ez indices
    elecmovedindices = []
    for ezindex in ezindices:
        print "Moving onto current ez index: ", ezindex, " at ", regions[ezindex]
         # find the closest contact index and distance
        seeg_index, distance = movecontact.findclosestcontact(ezindex, elecmovedindices)
        
        # get the modified seeg xyz and gain matrix
        modseeg, electrodeindices = movecontact.movecontact(ezindex, seeg_index)
        elecmovedindices.append(electrodeindices)
        
    modgain = movecontact.simplest_gain_matrix(movecontact.seeg_xyz)
    
    ####################### 2. Neural Mass Model @ Nodes ######################
    epileptors = initepileptor(epileptor_r, epiks, epitt, epitau, x0norm, \
                              x0ez, x0pz, ezindices, pzindices, num_regions)    
    ####################### 3. Integrator for Models ##########################
    heunint = initintegrator(heun_ts, noise_cov)
    ################## 4. Difference Coupling Between Nodes ###################
    coupl = initcoupling(a=1.)
    ############## 5. Import Sensor XYZ, Gain Matrix For Monitors #############
    monitors = initmonitors(period, sensorsfile, gainmatfile, varindex)
    
    if MOVECONTACT:
        print "moving contacts for ", patient
        # modify the config of the monitors
        monitors[1].sensors.locations = movecontact.seeg_xyz
        monitors[1].gain = modgain
    
    # get initial conditions and then setup entire simulation configuration
    initcond = initconditions(x0norm, num_regions)
    sim, configs = setupconfig(epileptors, con, coupl, heunint, monitors, initcond)
    
    times, epilepts, seegts = runsim(sim, sim_length)

    postprocessor = PostProcess(epilepts, seegts, times)
    ######################## POST PROCESSING #################################
    # post process by cutting off first 5 seconds of simulation
    # for now, don't, since intiial conditions
    times, epits, seegts, zts = postprocessor.postprocts(samplerate)

    # get the onset, offset times
    onsettimes = None
    offsettimes = None
    try:
        onsettimes, offsettimes = PostProcess.findonsetoffset(zts[ezindices, :].squeeze(), delta=0.2)
    except:
        print "Still not working..."

    ######################## SAVING ALL DATA #################################
    regions = configs.connectivity.region_labels
    # Save files
    meta = {
        'x0ez':x0ez,
        'x0pz':x0pz,
        'x0norm':x0norm,
        'regions': regions,
        'regions_centers': configs.connectivity.centres,
        'seeg_contacts': configs.monitors[1].sensors.labels,
        'seeg_xyz': configs.monitors[1].sensors.locations,
        'ez': regions[ezindices],
        'pz': regions[pzindices],
        'ezindices': ezindices,
        'pzindices': pzindices,
        'onsettimes':onsettimes,
        'offsettimes':offsettimes,
        'patient':patient,
    }

    # save tseries
    np.savez(filename, epits=epits, seegts=seegts, \
             times=times, zts=zts, metadata=meta)

# 2. Apply EZ To Closest Region

Here, set everything from the ez region, connectivity, node model, coupling, monitors, integrator, and parameters.

In [ ]:
# patient = 'id014_rb'
patients = ['id001_ac', 'id002_cj', 'id014_rb']

# 1000 = 1 second
samplerate = 1000 # Hz
sim_length = 240*samplerate

######### Region Parameters ##########
ezregion = ['ctx-lh-bankssts', 'ctx-lh-cuneus']
ezregion = ['ctx-lh-bankssts']
ezregion = regions[minregion]
#     ezregion = []
pzregion = ['ctx-lh-cuneus']
# pzregion = [] 

######### Epileptor Parameters ##########
period = 1
# x0c value = -2.05
x0norm=-2.5
x0ez=-1.6
x0pz=-2.0
# intialized hard coded parameters
epileptor_r = 0.0002       # Temporal scaling in the third state variable
epiks = -0.5                 # Permittivity coupling, fast to slow time scale
epitt = 0.05               # time scale of simulation
epitau = 10                # Temporal scaling coefficient in fifth st var

# depends on epileptor variables of interest: it is where the x2-y2 var is
varindex = [1]

######### Integrator Parameters ##########
# parameters for heun-stochastic integrator
heun_ts = 0.05
noise_cov = np.array([0.001, 0.001, 0.,\
                0.0001, 0.0001, 0.])

root_dir = os.path.join('/home/adamli/metadata/')
resultsdir = os.path.join('/home/adamli/data/tvbforwardsim/')

############## Loop Thru Patients ##############
for patient in patients:
    project_dir = os.path.join(root_dir, patient)
    tvbsim.util.renamefiles(patient, project_dir)

    ####### Initialize files needed to 
    sensorsfile = os.path.join(project_dir, "seeg.txt")
    confile = os.path.join(project_dir, "connectivity.zip")
    gainmatfile = os.path.join(project_dir, "gain_inv-square.txt")

    # extract connectivity 
    con = initconn(confile)

    # extract the seeg_xyz coords and the region centers
    seeg_xyz = tvbsim.util.extractseegxyz(sensorsfile)
    seeg_labels = seeg_xyz.index.values

    # extract the seeg_xyz coords and the region centers
    region_centers = con.centres
    regions = con.region_labels
    num_regions = len(regions)

    # initialize object to assist in moving seeg contacts
    movecontact = tvbsim.util.MoveContacts(seeg_labels, seeg_xyz, 
                                       regions, region_centers, True)
    ezindices = movecontact.getindexofregion(ezregion)
    pzindices = movecontact.getindexofregion(pzregion)

    # find the closest region-contact pair
    for idx, label in enumerate(seeg_labels):
        region_index, distance = movecontact.getregionsforcontacts(label)

        if idx == 0:
            mindist = distance
            minregion = region_index
            mincontact = label
        else:
            if distance < mindist:
                mindist = distance
                minregion = region_index
                mincontact = label
    
    outputdir = os.path.join(resultsdir, patient)
    if not os.path.exists(outputdir):
        os.mkdir(outputdir)
    filename = os.path.join(outputdir, patient+'closeregion_TESTTESTsim_nez'+str(len(ezregion))+\
                                        '_npz'+str(len(pzregion))+'.npz')

    print patient
    print "Project directory for meta data is : ", project_dir
    print "File to be saved is: ", filename
    print mincontact
    print mindist
    print minregion
    print regions[minregion]
    print patient, " will be simulated for ", sim_length
    print ezregion
    print "output dir is: ", outputdir
    print filename

    ####################### 1. Structural Connectivity ########################
    con = initconn(confile)
    ####################### 2. Neural Mass Model @ Nodes ######################
    epileptors = initepileptor(epileptor_r, epiks, epitt, epitau, x0norm, \
                              x0ez, x0pz, ezindices, pzindices, num_regions)    
    ####################### 3. Integrator for Models ##########################
    heunint = initintegrator(heun_ts, noise_cov)
    ################## 4. Difference Coupling Between Nodes ###################
    coupl = initcoupling(a=1.)
    ############## 5. Import Sensor XYZ, Gain Matrix For Monitors #############
    monitors = initmonitors(period, sensorsfile, gainmatfile, varindex)

    # get initial conditions and then setup entire simulation configuration
    initcond = initconditions(x0norm, num_regions)
    sim, configs = setupconfig(epileptors, con, coupl, heunint, monitors, initcond)

    times, epilepts, seegts = runsim(sim, sim_length)
    
    ######################## POST PROCESSING #################################
    postprocessor = tvbsim.util.PostProcess(epilepts, seegts, times)
    # post process by cutting off first 5 seconds of simulation
    # for now, don't, since intiial conditions
    times, epits, seegts, zts = postprocessor.postprocts(samplerate)

    # get the onset, offset times
    onsettimes = None
    offsettimes = None
    try:
        onsettimes, offsettimes = postprocessor.findonsetoffset(zts[ezindices, :].squeeze(), delta=0.2)
    except:
        print "Still not working..."

    ######################## SAVING ALL DATA #################################
    regions = configs.connectivity.region_labels
    # Save files
    meta = {
        'x0ez':x0ez,
        'x0pz':x0pz,
        'x0norm':x0norm,
        'regions': regions,
        'regions_centers': configs.connectivity.centres,
        'seeg_contacts': configs.monitors[1].sensors.labels,
        'seeg_xyz': configs.monitors[1].sensors.locations,
        'ez': regions[ezindices],
        'pz': regions[pzindices],
        'ezindices': ezindices,
        'pzindices': pzindices,
        'onsettimes':onsettimes,
        'offsettimes':offsettimes,
        'patient':patient,
    }

    # save tseries
    np.savez(filename, epits=epits, seegts=seegts, \
             times=times, zts=zts, metadata=meta)